### `MolecularTrainer`

该模块实现了分子 GCN 模型的训练循环、评估指标和模型检查点。

#### 一、`__init__(self, model, train_loader, val_loader, test_loader, device, experiment_dir, experiment_name)`

1. 设备管理

```python
if device == "auto":
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
```

- 自动检测并使用可用的 GPU，大大简化了使用

2. 实验管理

```python
self.experiment_name = experiment_name or f"experiment_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(self.experiment_path, exist_ok=True)
```

为每一次训练创建一个独立的、带时间戳的文件夹（如 `experiments/experiment_20251101_140000`），优势：
- **不会覆盖结果**：每次运行都会产生一个新的实验记录。
- **易于管理**：所有与该次实验相关的文件（模型、配置、图表）都会保存在同一个地方

3. 状态追踪

```python
self.train_history = { ... }
self.best_val_loss = float("inf")
self.patience_counter = 0
```

初始化了用于记录训练过程和实现早停的所有状态变量。`train_history` 字典会记录每个 `epoch` 的各种指标，为后续分析和绘图做准备

4. 配置保存

```python
self._save_config()
```

在训练开始前将模型类型、超参数、数据集大小、设备等关键信息保存为 `config.json` 文件。这保证了实验的**可复现性**。任何人都可以通过这个文件了解这次实验的确切配置

#### 二、设置方法用于配置训练组件

- `setup_optimizer`：允许用户通过字符串（`adam`, `adamw`, `sgd`）轻松选择优化器，并配置学习率、权重衰减等参数。
- `setup_scheduler`：同样，允许用户选择学习率调度策略。`ReduceLROnPlateau`（验证损失不再下降时自动降低学习率）是一个非常实用和强大的选择

#### 三、训练与评估逻辑

- `train_epoch`: 
    - `self.model.train()`：将模型设置为训练模式，激活 `Dropout` 和 `BatchNorm`
    - `optimizer.zero_grad() -> loss.backward() -> optimizer.step()`：标准的三步走
    - `torch.nn.utils.clip_grad_norm_`: 重要细节，用于梯度裁剪，防止梯度爆炸
    - **指标累积**：它没有在每个 batch 上计算最终指标，而是将所有 batch 的预测和目标收集起来，在整个 epoch 结束后统一计算。这比平均每个 batch 的指标更准确
- `validate_epoch`: 
    - `self.model.eval()`: 将模型设置为评估模式，禁用 `Dropout`
    - `with torch.no_grad()`: 关闭梯度计算，节省内存和计算资源
    - 其余逻辑与 `train_epoch` 类似，但没有反向传播和优化步骤

#### 四、`train` 方法

1. `Epoch` 循环：遍历指定的 num_epochs
2. 训练与验证：每个 `epoch` 都先调用 `train_epoch`，然后调用 `validate_epoch`
3. 学习率调度：根据调度器类型（`ReduceLROnPlateau`）传入验证损失来调整学习率
4. 历史记录：将每个 `epoch` 的所有指标存入 `self.train_history`
5. 早停：
    - 检查验证损失是否有显著改善（`< self.best_val_loss - min_delta`）
    - 如果有改善，更新最佳损失、重置耐心计数器，并标记 `is_best=True`
    - 如果没有，耐心计数器加一
    - 如果计数器超过 `patience`，则提前终止训练，防止过拟合
6. 检查点保存：
    - 定期保存（`save_every`）一个 `latest_checkpoint.pth`，用于随时恢复训练
    - 当发现新的最佳模型时（`is_best=True`），额外保存一个 `best_model.pth`，用于最终部署
7. 日志打印：非常详细地打印每个 `epoch` 的关键信息，方便实时监控
8. 最终测试与可视化：训练结束后，如果提供了测试集，会进行最终评估，并调用 `_plot_training_curves` 生成训练曲线图

#### 五、辅助方法

- `save_checkpoint` / `load_checkpoint`: 实现了模型的保存和加载。保存的内容包括模型权重、优化器状态、调度器状态、最佳 epoch 等信息，实现训练的无缝中断和恢复
- `_plot_training_curves`: 自动生成包含损失、RMSE、R² 和学习率变化的 2x2 子图，并保存为高清图片。这对于分析模型行为、诊断问题（如过拟合、学习率不当）至关重要